In [2]:
%load_ext autoreload
%autoreload 2
%env ANYWIDGET_DEV=1

env: ANYWIDGET_DEV=1


In [3]:
import numpy as np
from sequilt import Sequilt
from sequilt.model import EventGraph, EventCanvas, SequletModel, LabelModel

In [67]:
from datasets import load_dataset
from sequilt.data import get_ids, get_tokenizer

ds = load_dataset("neuralbioinfo/bacterial_promoters")
tokenizer = get_tokenizer(type="dna", k=1)
ids, tokens = get_ids(
  ds["test_multispecies"]["segment"], tokenizer, max_tokens=32
)
labels = [
  LabelModel(value=value, name=name)
  for value, name in tokenizer._id_to_token.items()
  if value != 0
]
labels = [
  labels[2], labels[1], labels[0], labels[3]
]

100%|██████████| 22582/22582 [00:00<00:00, 30451.70it/s]


In [102]:
from datasets import load_dataset
from sequilt.data import get_ids, get_tokenizer, get_featured_ids

ds = load_dataset("ajaykarthick/imdb-movie-reviews")
tokenizer = get_tokenizer(type="language")
ids, tokens = get_ids(
  ds['test']["review"], tokenizer, max_tokens=32
)
featured_ids = get_featured_ids(ids, tokenizer, "tf-idf", n_features=10)
labels = [
  LabelModel(value=id, name=tokenizer.id_to_token(id))
  for id in featured_ids
]

100%|██████████| 10000/10000 [00:07<00:00, 1316.03it/s]


In [103]:
feature_mask = np.isin(ids, featured_ids)
print(ids.shape, feature_mask.shape)

(10000, 32) (10000, 32)


In [104]:
ids_lang = np.where(feature_mask, ids, 0)
ids_lang.shape

(10000, 32)

In [107]:
G = EventGraph(ids_lang)

canvas = EventCanvas(width=ids.shape[1], initial_height=ids.shape[0] * 2)

for event1, event2, cooccurence in G.sorted_edges:
  sequlet = SequletModel([event1, event2])
  canvas.draw_sequlet(sequlet)
  G.remove_events_from([event1, event2])

In [106]:
for node in G.nodes:
  sequlet = SequletModel([node])
  canvas.draw_sequlet(sequlet)

KeyboardInterrupt: 

In [108]:
w = Sequilt(sequlets=canvas.sequlet_rects, labels=labels, width=800, height=800)
w

Sequilt(height=800, labels=[{'value': 58, 'name': 'this'}, {'value': 101, 'name': 'movie'}, {'value': 90, 'nam…